In [1]:
import pandas as pd
import re

# 1. Đọc dữ liệu
df = pd.read_excel(r'../Datasets/data_science_jobs.xlsx')

# --- BỔ SUNG: KHỞI TẠO DANH SÁCH TỪ DỪNG ---
# Bạn có thể tải file stopwords tiếng Việt hoặc dùng danh sách cơ bản dưới đây
stop_words_vi = ["và", "của", "là", "các", "cho", "trong", "được", "với", "như", "có", "một", "những", "tại", "này"]
stop_words_en = ["and", "the", "of", "to", "for", "with", "is", "in", "on", "at", "by", "an", "this"]
all_stopwords = set(stop_words_vi + stop_words_en)

def clean_text(text):
    if pd.isna(text): return ""
    
    # Chuyển về chữ thường để khớp stopword
    text = text.lower()
    
    # Xử lý các ký tự xuống dòng và khoảng trắng thừa
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)
    
    # Loại bỏ các ký tự đặc biệt (giữ lại chữ cái, số và khoảng trắng)
    text = re.sub(r'[^\w\s]', '', text)
    
    # --- BỔ SUNG: LOẠI BỎ TỪ DỪNG ---
    words = text.split()
    words = [w for w in words if w not in all_stopwords]
    text = ' '.join(words)
    
    return text.strip()

# 2. Kết hợp Mô tả và Yêu cầu
df['full_description'] = df['Mô tả công việc'].fillna('') + " " + df['Yêu cầu'].fillna('')
df['content_clean'] = df['full_description'].apply(clean_text)

# 3. Lọc bỏ các dòng không có nội dung hoặc quá ngắn sau khi xóa stopword
df = df[df['content_clean'].str.len() > 100]

# 4. Gán nhãn cho E5
df['e5_input'] = "passage: " + df['content_clean']


# xử lý cột Chuyên môn, Thời gian đăng để phân tích

In [2]:
print(df['Chuyên môn'].value_counts())

Chuyên môn
Chức vụ: Nhân viên                     610
data_analyst                           384
AI / Machine Learning Engineer          81
data_scientist                          57
Chức vụ: Quản lý                        40
ml_engineer                             36
Data Scientist                          34
Data Engineer                           26
Data Analyst                            21
Chức vụ: Thực tập sinh                  19
Backend Developer                       14
Business Analyst                        11
Chức vụ: Lãnh đạo                        4
Computer Vision Engineer                 4
Hạn nộp: Còn 15 ngày                     4
Hạn nộp: Còn 11 ngày                     4
Product Manager                          4
Hạn nộp: Còn 1 ngày                      3
Hạn nộp: Còn 42 ngày                     3
Hạn nộp: Còn 16 ngày                     3
Test Coordinator / QAQC Coordinator      2
Systems Engineer / Administrator         2
AI Architect                             2


In [3]:
#Hàm chuẩn hóa chuyên môn
def map_specialization(job_title):
    job_title = str(job_title).lower()
    if any(x in job_title for x in ['ml_engineer', 'machine learning', 'ai', 'computer vision', 'ml engineer',
                                    'ai / machine learning engineer','ai architect','computer vision engineer',
                                    ]):
        return 'ML Engineer'
    elif any(x in job_title for x in ['data_scientist', 'data scientist', 'science','database engineer',
                                      'big data engineer']):
        return 'Data Scientist'
    elif any(x in job_title for x in ['data_analyst', 'data analyst', 'business analyst', 'ba']):
        return 'Data Analyst'
    else:
        return 'Other' # Các nhóm không thuộc 3 nhóm trên
    
df['Chuyên môn'] = df['Chuyên môn'].apply(map_specialization)

print(df['Chuyên môn'].value_counts())

Chuyên môn
Other             2710
Data Analyst       432
ML Engineer        123
Data Scientist      93
Name: count, dtype: int64


In [4]:
df['Thời gian đăng'].value_counts()

Thời gian đăng
1 week ago     266
2 weeks ago    202
3 weeks ago    112
3 days ago      84
5 days ago      80
              ... 
06/12/2025       1
22/11/2025       1
29/11/2025       1
13/12/2025       1
9 days ago       1
Name: count, Length: 64, dtype: int64

In [5]:
from datetime import datetime, timedelta

# Thiết lập mốc thời gian thu thập
collection_date = datetime(2025, 12, 24)

def parse_posted_date(date_str):
    if pd.isna(date_str):
        return None
    
    date_str = str(date_str).lower().strip()
    
    try:
        # Trường hợp 1: Dạng ngày tuyệt đối (DD/MM/YYYY)
        if '/' in date_str:
            return datetime.strptime(date_str, '%d/%m/%Y')
        
        # Trường hợp 2: Dạng tương đối (days ago, weeks ago)
        number = int(re.search(r'\d+', date_str).group())
        
        if 'day' in date_str:
            return collection_date - timedelta(days=number)
        elif 'week' in date_str:
            return collection_date - timedelta(weeks=number)
        elif 'month' in date_str:
            return collection_date - timedelta(days=number * 30) # Ước tính 30 ngày/tháng
        elif 'hour' in date_str:
            return collection_date # Coi như cùng ngày thu thập
            
    except Exception:
        return None
    return None

# Áp dụng hàm chuyển đổi
df['Ngày đăng chuẩn'] = df['Thời gian đăng'].apply(parse_posted_date)

# Kiểm tra kết quả
print(df[['Thời gian đăng', 'Ngày đăng chuẩn']].head(20))

   Thời gian đăng Ngày đăng chuẩn
0      20/11/2025      2025-11-20
1      18/12/2025      2025-12-18
2      20/11/2025      2025-11-20
3      18/12/2025      2025-12-18
4      17/12/2025      2025-12-17
5      04/12/2025      2025-12-04
6      15/12/2025      2025-12-15
7      26/11/2025      2025-11-26
8      15/12/2025      2025-12-15
9      09/12/2025      2025-12-09
10     19/11/2025      2025-11-19
11     16/12/2025      2025-12-16
12     19/11/2025      2025-11-19
13     08/12/2025      2025-12-08
14     10/12/2025      2025-12-10
15     10/12/2025      2025-12-10
16     15/12/2025      2025-12-15
17     12/12/2025      2025-12-12
18     24/11/2025      2025-11-24
19     26/11/2025      2025-11-26


In [6]:
df['Thời gian đăng'] = df['Thời gian đăng'].apply(parse_posted_date)
df = df.drop('Ngày đăng chuẩn', axis=1)

In [ ]:
# Lưu file đã làm sạch
df.to_csv(r"../Datasets/jobs_cleaned.csv", index=False, encoding="utf-8-sig")
print(f" Đã xử lý và loại bỏ từ dừng cho {len(df)} bản ghi.")

 Đã xử lý và loại bỏ từ dừng cho 3358 bản ghi.
